In [82]:
# import Depencies
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import numpy as np

import regex
from collections import Counter


In [83]:
# Set header to circumvent webscraping blocker
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

In [84]:
# Create lists to store **responses**
job_title = []
company_name = []
short_description = []
job_location = []
job_classification = []
job_subclassification = []
job_site = []

job_counter = 0

In [85]:
headers = {
     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [86]:
# Scrape the website
# Create list of scraping variables (job fields, pages[4])
job_fields = ['Data-Analyst', 'Data-Engineer']
page_scrapes = ['', '?page=2', '?page=3','?page=4','?page=5']

In [87]:
# Scrape data
for job_field in job_fields:
    for page in page_scrapes:
        url = f'https://www.seek.com.au/{job_field}-jobs/in-All-Australia{page}'
        
        # Make a request to the URL
        response = requests.get(url, headers=headers)
        
        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to retrieve data from {url}")
            continue
        
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        jobs_on_page = soup.find_all('article')

        # Process each job on the page
        for job in jobs_on_page:
            job_counter += 1
            
            # Extract job details
            try:
                title = job.get('aria-label', 'N/A')
                job_title.append(title)
            except AttributeError:
                job_title.append('N/A')
                
            job_site.append("Seek.com.au")
            
            try:
                desc = job.find('span', {'data-automation': 'jobShortDescription'}).text
                short_description.append(desc)
            except AttributeError:
                short_description.append('N/A')
            
            try:
                location = job.find('a', {'data-automation': 'jobLocation'}).text
                job_location.append(location)
            except AttributeError:
                job_location.append('N/A')
            
            try:
                sub_class = job.find('a', {'data-automation': 'jobSubClassification'}).text
                job_subclassification.append(sub_class)
            except AttributeError:
                job_subclassification.append('N/A')
            
            try:
                classification = job.find('a', {'data-automation': 'jobClassification'}).text
                job_classification.append(classification)
            except AttributeError:
                job_classification.append('N/A')
            
            try:
                company = job.find('a', {'data-automation': 'jobCompany'}).text
                company_name.append(company)
            except AttributeError:
                company_name.append(np.nan)
            
            print(f'Jobs scraped: {job_counter}', end='\r')

-------



# end of scrape


In [88]:
# output the number of characters in each of the variables
print(len(company_name))
print(len(job_title))
print(len(short_description))

220
220
220


In [89]:
# dictionary named seek_data to store various pieces of information about a job
seek_data = {'job_title': job_title,
            'company_name': company_name,
            'short_description': short_description,
            'job_location': job_location,
            'job_classification': job_classification,
            'job_subclassification': job_subclassification,
            'job_site' : job_site
             }

In [90]:
# create a DataFrame
seek_data_df = pd.DataFrame(seek_data)
# reset Index:
seek_data_df.reset_index(inplace=True)
# rename Index Column
seek_data_df.rename(columns= {'index':'job_id'}, inplace=True)
# display the Last 150 Rows
seek_data_df.tail(150)

,job_id,job_title,company_name,short_description,job_location,job_classification,job_subclassification,job_site
70,70,Data & Reporting Analyst,Sexual Health Victoria,"Lead the collection, analysis and reporting of...",Box Hill,(Information & Communication Technology),Business/Systems Analysts,Seek.com.au
71,71,Data Analyst,Hapana,Collaborate with a diverse team to develop the...,Sydney NSW,(Information & Communication Technology),Database Development & Administration,Seek.com.au
72,72,Business/Data Analyst - Telematics Fleet Manag...,IntelliTrac,An exciting opportunity for a business/data an...,Bundoora,"(Manufacturing, Transport & Logistics)",Analysis & Reporting,Seek.com.au
73,73,Online Data Analyst - Australia,TELUS International AI INC,TELUS International AI-Data Solutions partners...,Brisbane QLD,(Information & Communication Technology),Other,Seek.com.au
74,74,Senior Data Analyst,Department of Education,The Senior Data Analyst will join a small dyna...,Melbourne VIC,(Information & Communication Technology),Business/Systems Analysts,Seek.com.au
...,...,...,...,...,...,...,...,...
215,215,Senior Database Administrator / Data Engineer,365 Assistance,Drive innovation at 365 Assistance! As a Senio...,Sydney NSW,(Information & Communication Technology),Database Development & Administration,Seek.com.au
216,216,Data Operations Engineer,Heritage and People's Choice,We are looking for a highly skilled Data Opera...,Adelaide SA,(Information & Communication Technology),Developers/Programmers,Seek.com.au
217,217,Data Engineer,National Heavy Vehicle Regulator,Data Engineer will contribute to improving reg...,Brisbane QLD,(Information & Communication Technology),Database Development & Administration,Seek.com.au
218,218,Data Engineer,Star Pharmacy Group,Star Pharmacy Group has an exciting opportunit...,Kent Town,(Information & Communication Technology),Database Development & Administration,Seek.com.au


In [91]:
# check the data types
print(seek_data_df.dtypes)

job_id                    int64
job_title                object
company_name             object
short_description        object
job_location             object
job_classification       object
job_subclassification    object
job_site                 object
dtype: object


In [92]:
# Function to remove specific characters like parentheses
def remove_parentheses(text):
    if isinstance(text, str):  # Ensure the input is a string
        text = text.replace('(', '')  # Remove opening parenthesis
        text = text.replace(')', '')  # Remove closing parenthesis
        return text
    return text

# Apply the function to each element in the DataFrame
seek_data_df_cleaned = seek_data_df.apply(lambda col: col.map(remove_parentheses) if col.dtype == 'object' else col)

# Display the cleaned DataFrame
seek_data_df_cleaned

,job_id,job_title,company_name,short_description,job_location,job_classification,job_subclassification,job_site
0,0,Data Analyst - 6 month contract - Hybrid,SEEK Limited,Work within the Growth Operations and Analyst ...,Cremorne,Consulting & Strategy,Analysts,Seek.com.au
1,1,Senior Data Analyst,Wesfarmers Health,Play a key role in shaping the future of healt...,Melbourne VIC,Information & Communication Technology,Other,Seek.com.au
2,2,Data Analyst - Policy,"Talent – Specialists in tech, transformation &...","Lead and support funding policy, complex model...",Melbourne VIC,Government & Defence,"Policy, Planning & Regulation",Seek.com.au
3,3,Junior Data Analyst,Randstad Digital,We are seeking a junior Data analyst on an ini...,Brisbane QLD,Information & Communication Technology,Other,Seek.com.au
4,4,National Data Analyst,Capital Transport VIC,We are seeking a National Data Analyst to join...,Clayton,"Manufacturing, Transport & Logistics",Analysis & Reporting,Seek.com.au
...,...,...,...,...,...,...,...,...
215,215,Senior Database Administrator / Data Engineer,365 Assistance,Drive innovation at 365 Assistance! As a Senio...,Sydney NSW,Information & Communication Technology,Database Development & Administration,Seek.com.au
216,216,Data Operations Engineer,Heritage and People's Choice,We are looking for a highly skilled Data Opera...,Adelaide SA,Information & Communication Technology,Developers/Programmers,Seek.com.au
217,217,Data Engineer,National Heavy Vehicle Regulator,Data Engineer will contribute to improving reg...,Brisbane QLD,Information & Communication Technology,Database Development & Administration,Seek.com.au
218,218,Data Engineer,Star Pharmacy Group,Star Pharmacy Group has an exciting opportunit...,Kent Town,Information & Communication Technology,Database Development & Administration,Seek.com.au


---

## Create dataframes for tables

---

In [93]:
# Create job_id dataframe

job_id_df = seek_data_df_cleaned[['job_id', 'job_classification', 'job_location']].copy()

# rename columns to match schema
job_id_df.rename(columns= {'job_classification':'job_field'}, inplace=True)

job_id_df

,job_id,job_field,job_location
0,0,Consulting & Strategy,Cremorne
1,1,Information & Communication Technology,Melbourne VIC
2,2,Government & Defence,Melbourne VIC
3,3,Information & Communication Technology,Brisbane QLD
4,4,"Manufacturing, Transport & Logistics",Clayton
...,...,...,...
215,215,Information & Communication Technology,Sydney NSW
216,216,Information & Communication Technology,Adelaide SA
217,217,Information & Communication Technology,Brisbane QLD
218,218,Information & Communication Technology,Kent Town


### Count words to plot - All Employment Fields

In [94]:
# Word Cloud Data

full_count = (seek_data_df['short_description'].str.split(expand=True)
              .stack())
full_count_df = pd.DataFrame(full_count)
full_count_df = full_count_df[full_count_df[0].str.len() > 4]
full_count_df

0
0   1        within
    3        Growth
    4    Operations
    6       Analyst
    7         Team,
...             ...
219 3      Brisbane
    4         based
    5       Federal
    6    Government
    7  opportunity.

[2080 rows x 1 columns]

In [95]:
# count, tally and describe words in short_description

seek_count = (seek_data_df['short_description'].str.split(expand=True)
              .stack()
              .value_counts()
              .rename_axis('word')
              .reset_index(name='count'))

In [96]:
seek_count = seek_count[seek_count['word'].str.len() > 4]
seek_count

,word,count
10,Engineer,48
14,Analyst,35
23,seeking,21
28,opportunity,16
29,business,16
...,...,...
1198,Register.,1
1199,Create,1
1200,Dashboards,1
1201,functions,1


In [97]:
seek_count.head(45)

,word,count
10,Engineer,48
14,Analyst,35
23,seeking,21
28,opportunity,16
29,business,16
30,contract,16
31,Senior,15
37,drive,13
39,within,12
40,critical,12


### Count words to plot - Individual fields (Data-Analyst, Data-Visualisation)

In [98]:
# Create DF for each job field

data_analyst_df = seek_data_df_cleaned[seek_data_df_cleaned['job_classification']=='Banking & Financial Services']
data_engineer_df = seek_data_df_cleaned[seek_data_df_cleaned['job_classification']=='Information & Communication Technology']

In [99]:
# create word count DF for each job field
# word cloud
# count, tally and describe words in short_description

# data_analyst

data_analyst_count = (data_analyst_df['short_description'].str.split(expand=True)
              .stack()
              .value_counts()
              .rename_axis('word')
              .reset_index(name='count'))

# data_engineer

data_engineer_count = (data_engineer_df['short_description'].str.split(expand=True)
              .stack()
              .value_counts()
              .rename_axis('word')
              .reset_index(name='count'))

In [100]:
# data_analyst

# Flatten the DataFrame, split into words, and remove non-word characters
data_analyst_set = data_analyst_df.stack().str.split(r'\W+', expand=False).explode().tolist()

# Convert to DataFrame
data_analyst_count_df = pd.DataFrame(data_analyst_set, columns=['Word'])
# Filter words with length greater than 4 characters
data_analyst_count_df = data_analyst_count_df[data_analyst_count_df['Word'].str.len() > 4]
# Count frequencies of each word
data_analyst_count_df = data_analyst_count_df['Word'].value_counts().reset_index()
data_analyst_count_df.columns = ['Word', 'Count']

# data_engineer

# Flatten the DataFrame, split into words, and remove non-word characters
data_engineer_set = data_engineer_df.stack().str.split(r'\W+', expand=False).explode().tolist()

# Convert to DataFrame
data_engineer_count_df = pd.DataFrame(data_engineer_set, columns=['Word'])
# Filter words with length greater than 4 characters
data_engineer_count_df = data_engineer_count_df[data_engineer_count_df['Word'].str.len() > 4]
# Count frequencies of each word
data_engineer_count_df = data_engineer_count_df['Word'].value_counts().reset_index()
data_engineer_count_df.columns = ['Word', 'Count']

# Display the result
data_engineer_count_df.head(40)

,Word,Count
0,Technology,189
1,Information,185
2,Communication,183
3,Engineer,148
4,Analyst,100
5,Business,65
6,Systems,58
7,Analysts,58
8,Senior,57
9,Sydney,57


In [101]:
# Convert the column to string type before applying string operations
data_analyst_count['word'] = data_analyst_count['word'].astype(str)
data_engineer_count['word'] = data_engineer_count['word'].astype(str)

In [102]:
# line filters to keep only the rows where the length of the string in the 'word' column is greater than 4
data_analyst_count = data_analyst_count[data_analyst_count['word'].str.len() > 4]
data_engineer_count = data_engineer_count[data_engineer_count['word'].str.len() > 4]

In [103]:
# print the result
print("Data Analyst Count:")
print(data_engineer_count)

Data Analyst Count:
             word  count
10       Engineer     47
19        Analyst     21
22        seeking     17
25       contract     15
26         Senior     14
...           ...    ...
1039       fleet.      1
1040    effective      1
1041  information      1
1043      oversee      1
1044    expanding      1

[832 rows x 2 columns]


### Count locations to plot

-----

In [104]:
# create df for location counts 
# heatmap plot

all_fields_locations = seek_data_df_cleaned['job_location'].value_counts()
data_analyst_locations = data_analyst_df['job_location'].value_counts()
data_engineer_locations = data_engineer_df['job_location'].value_counts()


all_fields_locations_df = pd.DataFrame(all_fields_locations)
data_analyst_locations_df = pd.DataFrame(data_analyst_locations)
data_engineer_locations_df = pd.DataFrame(data_engineer_locations)

data_engineer_locations

job_location
Sydney NSW             50
Melbourne VIC          24
Canberra ACT           23
Brisbane QLD           23
Perth WA                7
Adelaide SA             4
Booragoon               3
Surry Hills             2
Geelong                 2
Parramatta              2
Belmont                 2
Gosford                 2
Hobart TAS              2
Artarmon                2
Hawthorn East           2
Griffith                2
Maroochydore            2
Alexandria              2
Broadbeach              1
Capital Hill            1
Collingwood             1
Burwood                 1
Osborne                 1
Richmond                1
Norwest                 1
North Ryde              1
Pyrmont                 1
Mascot                  1
Enoggera                1
Annerley                1
Exton                   1
Burwood East            1
Henderson               1
Sunshine Coast QLD      1
Box Hill                1
Mulgrave                1
Dandenong               1
Macquarie Park          1

In [105]:
# reset index for tables
all_fields_locations_df.reset_index(inplace=True)
data_analyst_locations_df.reset_index(inplace=True)
data_engineer_locations_df.reset_index(inplace=True)


In [106]:
# rename columns to be more friendly
all_fields_locations_df.rename(columns= {'index':'job_location', 'job_location':'count'}, inplace=True)
data_analyst_locations_df.rename(columns= {'index':'job_location', 'job_location':'count'}, inplace=True)
data_engineer_locations_df.rename(columns= {'index':'job_location', 'job_location':'count'}, inplace=True)

data_engineer_locations_df

,count,count
0,Sydney NSW,50
1,Melbourne VIC,24
2,Canberra ACT,23
3,Brisbane QLD,23
4,Perth WA,7
5,Adelaide SA,4
6,Booragoon,3
7,Surry Hills,2
8,Geelong,2
9,Parramatta,2


------

### Connect to postgres and load data to db

-----

In [107]:
# MOVE TO TOP BEFORE SUBMITTING
# from api_keys import postgres_p
from sqlalchemy import create_engine
from sqlalchemy import inspect
import psycopg2

In [108]:
# Define the connection parameters
protocol = 'postgresql'
username = 'postgres'
password = "Jane2801kei!"

host = 'localhost'

port = 5432

database_name = 'Project_3_1'

# Create the connection string
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
# Create an engine
engine = create_engine(rds_connection_string)
# Connect to the database
insp = inspect(engine)

insp.get_table_names()

['job_id',
 'data_analyst_scrapes',
 'data_engineer_scrapes',
 'data_analyst_word_count',
 'data_engineer_word_count']

In [109]:
data_engineer_df

,job_id,job_title,company_name,short_description,job_location,job_classification,job_subclassification,job_site
1,1,Senior Data Analyst,Wesfarmers Health,Play a key role in shaping the future of healt...,Melbourne VIC,Information & Communication Technology,Other,Seek.com.au
3,3,Junior Data Analyst,Randstad Digital,We are seeking a junior Data analyst on an ini...,Brisbane QLD,Information & Communication Technology,Other,Seek.com.au
7,7,Data Quality Analyst,Randstad Digital,Seeking a Data Quality Analyst ready to take t...,Melbourne VIC,Information & Communication Technology,Business/Systems Analysts,Seek.com.au
8,8,Data Analyst | Digital Media Products,ABC,Use audience data to generate insights that ad...,Sydney NSW,Information & Communication Technology,Business/Systems Analysts,Seek.com.au
9,9,Data Analyst | Digital Media Products,ABC,Use audience data to generate insights that ad...,Brisbane QLD,Information & Communication Technology,Business/Systems Analysts,Seek.com.au
...,...,...,...,...,...,...,...,...
214,214,Senior Data Engineer :: Syd/Melb :: $1100pd in...,Correlate Resources,Correlate Resources: Your trusted Data Analyti...,Sydney NSW,Information & Communication Technology,Developers/Programmers,Seek.com.au
215,215,Senior Database Administrator / Data Engineer,365 Assistance,Drive innovation at 365 Assistance! As a Senio...,Sydney NSW,Information & Communication Technology,Database Development & Administration,Seek.com.au
216,216,Data Operations Engineer,Heritage and People's Choice,We are looking for a highly skilled Data Opera...,Adelaide SA,Information & Communication Technology,Developers/Programmers,Seek.com.au
217,217,Data Engineer,National Heavy Vehicle Regulator,Data Engineer will contribute to improving reg...,Brisbane QLD,Information & Communication Technology,Database Development & Administration,Seek.com.au


In [110]:
# load to database

# Job_ID Table

job_id_df.to_sql(name='job_id', con=engine, if_exists='append', index=False)

## Scraped job Tables

data_analyst_df.to_sql(name='data_analyst_scrapes', con=engine, if_exists='append', index=False)
data_engineer_df.to_sql(name='data_engineer_scrapes', con=engine, if_exists='append', index=False)

183

In [111]:
pd.read_sql_query('select * from data_analyst_scrapes', con=engine).head(15)

,job_id,job_title,company_name,short_description,job_location,job_classification,job_subclassification,job_site
0,20,Data Analyst,"Talent – Specialists in tech, transformation &...","Our client is a leading financial institution,...",Sydney NSW,Banking & Financial Services,Analysis & Reporting,Seek.com.au
1,21,Data Analyst,CBHS Health Fund Limited,Our members are at the heart of everything we ...,Parramatta,Banking & Financial Services,Analysis & Reporting,Seek.com.au
2,23,Data Analyst,CBHS Health Fund Limited,Our members are at the heart of everything we ...,Parramatta,Banking & Financial Services,Analysis & Reporting,Seek.com.au
3,27,Data Analyst,McMillan Shakespeare,Bring your data analysis skills to conduct ana...,Melbourne VIC,Banking & Financial Services,Analysis & Reporting,Seek.com.au
4,44,Data Analyst,CBHS Health Fund Limited,Our members are at the heart of everything we ...,Parramatta,Banking & Financial Services,Analysis & Reporting,Seek.com.au
5,64,Investment Data Analyst,CBUS Super,Come be a part of team that focuses on deliver...,Melbourne VIC,Banking & Financial Services,Analysis & Reporting,Seek.com.au


In [112]:
# ## Word counts

data_analyst_count.to_sql(name='data_analyst_word_count', con=engine, if_exists='append', index=False)
data_engineer_count.to_sql(name='data_engineer_word_count', con=engine, if_exists='append', index=False)

832

In [113]:
pd.read_sql_query('select * from data_engineer_word_count', con=engine).head(15)

,word,count
0,Engineer,47
1,Analyst,21
2,seeking,17
3,contract,15
4,Senior,14
5,opportunity,13
6,critical,12
7,looking,11
8,business,10
9,Azure,10
